In [1]:
import ollama
from ollama import generate
import glob
import pandas as pd
from PIL import Image

import os
from io import BytesIO

In [2]:
# Load the DataFrame from a CSV file, or create a new one if the file doesn't exist
def load_or_create_dataframe(filename):
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        df = pd.DataFrame(columns=['image_file', 'description'])
    return df

df = load_or_create_dataframe('image_descriptions.csv')

def get_files(folder_path):
    return glob.glob(f"{folder_path}/*.jpg")

In [3]:
# get the list of image files in the folder yopu want to process
image_files = get_files("/home/tom/Python/OCR/images/") 
image_files.sort()

print(image_files[:3])
print(df.head())

['/home/tom/Python/OCR/images/PXL_20220512_211750155-2.jpg', '/home/tom/Python/OCR/images/PXL_20220512_211750155.jpg', '/home/tom/Python/OCR/images/PXL_20220512_214052210-2.jpg']
                                          image_file  \
0  /home/tom/Python/OCR/images/PXL_20220512_21175...   
1  /home/tom/Python/OCR/images/PXL_20220512_21175...   
2  /home/tom/Python/OCR/images/PXL_20220512_21405...   
3  /home/tom/Python/OCR/images/PXL_20220512_21405...   
4  /home/tom/Python/OCR/images/PXL_20220512_21443...   

                                         description  
0   The image shows a man taking a selfie in a ba...  
1   The image shows a person standing in front of...  
2   The image features a person taking a selfie i...  
3   In the image, there are two distinct photos p...  
4   The image is a photograph of an individual st...  


In [ ]:
!ollama pull llava

In [4]:
# processing the images 
def process_image(image_file):
    print(f"\nProcessing {image_file}\n")
    with Image.open(image_file) as img:
        with BytesIO() as buffer:
            img.save(buffer, format='JPEG')
            image_bytes = buffer.getvalue()

    full_response = ''
    # Generate a description of the image
    for response in generate(model='llava', 
                             prompt='describe this image and make sure to include anything notable about it (include text you see in the image):', 
                             images=[image_bytes], 
                             stream=True):
        # Print the response to the console and add it to the full response
        print(response['response'], end='', flush=True)
        full_response += response['response']

    # Add a new row to the DataFrame
    df.loc[len(df)] = [image_file, full_response]

In [5]:
for image_file in image_files:
    if image_file not in df['image_file'].values:
        process_image(image_file)

# Save the DataFrame to a CSV file
df.to_csv('image_descriptions.csv', index=False)